In [1]:
%load_ext rpy2.ipython

In [2]:
%%R
library(mltools)
library(data.table)
library(dplyr)
library(ggplot2)
library(ggiraph)
library('mclust')

IN_DIR <- '/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/tcdd/output/'
OUT_DIR <- '/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/tcdd/validation/'

gnn_data <- read.csv(paste0(IN_DIR, 'predictions030.tsv'), sep = "\t", row.names = NULL)
# resnet_data <- read.csv(paste0(IN_DIR, 'resnet_predictions.tsv'), sep = "\t", row.names = NULL)

data.table 1.15.4 using 24 threads (see ?getDTthreads).  Latest news: r-datatable.com

Attaching package: ‘dplyr’

The following objects are masked from ‘package:data.table’:

    between, first, last

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

                   __           __ 
   ____ ___  _____/ /_  _______/ /_
  / __ `__ \/ ___/ / / / / ___/ __/
 / / / / / / /__/ / /_/ (__  ) /_  
/_/ /_/ /_/\___/_/\__,_/____/\__/   version 6.1.1
Type 'citation("mclust")' for citing this R package in publications.


In [3]:
%%R
gnn_ari <- adjustedRandIndex(gnn_data$target, gnn_data$prediction)
print(paste0("gnn_ari: ", gnn_ari))

[1] "gnn_ari: 0.893397689896596"


In [4]:
%%R
print(gnn_data)

    project_ids sample_ids genders doses target prediction confidence_class_0
1     SRP161461 SRR7817611    male     0      0          0       9.999957e-01
2     SRP161461 SRR7817612    male     0      0          0       9.999933e-01
3     SRP161461 SRR7817614    male     0      0          0       9.999912e-01
4     SRP161461 SRR7817615    male     0      0          0       9.999683e-01
5     SRP161461 SRR7817616    male     0      0          0       9.999743e-01
6     SRP161461 SRR7817617    male    30      1          1       9.695386e-04
7     SRP161461 SRR7817618    male    30      1          1       7.588602e-04
8     SRP161461 SRR7817619    male    30      1          1       3.792945e-05
9     SRP161461 SRR7817620    male    30      1          1       3.987029e-04
10    SRP161461 SRR7817621    male    30      1          1       6.284971e-03
11    SRP161461 SRR7817622    male    30      1          1       5.762082e-03
12    SRP161461 SRR7817623    male     0      0          0      

105   SRP049440 SRR1636590  female     0      0          0       9.735683e-01
106   SRP049440 SRR1636671  female    30      1          1       6.943703e-06
107   SRP049440 SRR1636672  female    30      1          1       8.781303e-05
108   SRP049440 SRR1636673  female    30      1          1       3.260613e-07
109   SRP049440 SRR1636674  female    30      1          1       4.476773e-06
110   SRP049440 SRR1636675  female    30      1          1       1.241732e-03
111   SRP049440 SRR1636676  female    30      1          1       1.782425e-05
112   SRP131784 SRR6653152    male     0      0          0       9.657007e-01
113   SRP131784 SRR6653153    male    30      1          1       4.591775e-01
114   SRP131784 SRR6653155    male    30      1          0       6.720106e-01
115   SRP131784 SRR6653157    male    30      1          1       9.884145e-02
116   SRP131784 SRR6653209    male     0      0          0       9.992257e-01
117   SRP131784 SRR6653210    male     0      0          0      

210   SRP090688 SRR4317607    male     0      0          0       9.583471e-01
211   SRP090688 SRR4317609    male     0      0          0       9.258755e-01
212   SRP090688 SRR4317610    male     0      0          0       9.477959e-01
213   SRP090688 SRR4317611    male     0      0          0       9.808934e-01
214   SRP090688 SRR4317612    male     0      0          0       9.446583e-01
215   SRP090688 SRR4317655    male    30      1          1       1.271276e-04
216   SRP090688 SRR4317656    male    30      1          1       2.604920e-04
217   SRP090688 SRR4317657    male    30      1          1       1.814540e-06
218   SRP090688 SRR4317658    male    30      1          1       5.878423e-06
219   SRP090688 SRR4317659    male    30      1          1       1.475699e-03
220   SRP090688 SRR4317660    male    30      1          1       3.408190e-04
    confidence_class_1
1         4.272521e-06
2         6.617382e-06
3         8.790515e-06
4         3.167512e-05
5         2.578655e-05
6   

In [6]:
%%R
unique_dose <- unique(c(gnn_data$target, gnn_data$prediction))
names <- unique(unique_dose)
gnn_results <- data.frame(names = names, dose = unique_dose)

gnn_results$TP <- 0
gnn_results$TN <- 0
gnn_results$FP <- 0
gnn_results$FN <- 0

for (dose in unique_dose) {
  for (i in 1:nrow(gnn_data)) {
    if (gnn_data$target[i] == dose) {
      if (gnn_data$prediction[i] == dose) {
        gnn_results[gnn_results$dose == dose, "TP"] <- gnn_results[gnn_results$dose == dose, "TP"] + 1
      } else {
        gnn_results[gnn_results$dose == dose, "FN"] <- gnn_results[gnn_results$dose == dose, "FN"] + 1
      }
    } else {
      if (gnn_data$prediction[i] == dose) {
        gnn_results[gnn_results$dose == dose, "FP"] <- gnn_results[gnn_results$dose == dose, "FP"] + 1
      } else {
        gnn_results[gnn_results$dose == dose, "TN"] <- gnn_results[gnn_results$dose == dose, "TN"] + 1
      }
    }
  }
}

gnn_results$POSITIVES <- gnn_results$TP + gnn_results$FN

gnn_results$MCC <- (gnn_results$TP * gnn_results$TN - gnn_results$FP * gnn_results$FN) / 
  sqrt((gnn_results$TP + gnn_results$FP) * (gnn_results$TP + gnn_results$FN) * (gnn_results$TN + gnn_results$FP) * (gnn_results$TN + gnn_results$FN))

gnn_results$TPR <- gnn_results$TP / (gnn_results$TP + gnn_results$FN)

gnn_results$FPR <- gnn_results$FP / (gnn_results$FP + gnn_results$TN)

gnn_results$TNR <- gnn_results$TN / (gnn_results$TN + gnn_results$FP)

gnn_results$FNR <- gnn_results$FN / (gnn_results$FN + gnn_results$TP)

gnn_results$Precision <- gnn_results$TP / (gnn_results$TP + gnn_results$FP)

gnn_results$Recall <- gnn_results$TP / (gnn_results$TP + gnn_results$FN)

gnn_results$F1_score <- 2 * (gnn_results$Precision * gnn_results$Recall)/(gnn_results$Precision + gnn_results$Recall)

print(gnn_results)

write.table(gnn_results, file = paste(OUT_DIR, "gnn_MCC_Result030.txt"), sep="\t", row.names = FALSE, quote = FALSE)

  names dose  TP  TN FP FN POSITIVES       MCC       TPR        FPR       TNR
1     0    0 108 106  4  2       110 0.9456109 0.9818182 0.03636364 0.9636364
2     1    1 106 108  2  4       110 0.9456109 0.9636364 0.01818182 0.9818182
         FNR Precision    Recall  F1_score
1 0.01818182 0.9642857 0.9818182 0.9729730
2 0.03636364 0.9814815 0.9636364 0.9724771
